In [1]:
import os
print(os.listdir('./train/'))

['Charlock', 'Shepherds Purse', 'Loose Silky-bent', 'Black-grass', 'Small-flowered Cranesbill', 'Common Chickweed', 'Cleavers', 'Maize', 'Scentless Mayweed', 'Fat Hen', 'Common wheat', 'Sugar beet']


In [2]:
import fnmatch
import os
import numpy as np
import pandas as pd
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
np.random.seed(21)

path = './train/'
train_label = []
train_img = []
label2num = {'Loose Silky-bent':0, 'Charlock':1, 'Sugar beet':2, 'Small-flowered Cranesbill':3,
             'Common Chickweed':4, 'Common wheat':5, 'Maize':6, 'Cleavers':7, 'Scentless Mayweed':8,
             'Fat Hen':9, 'Black-grass':10, 'Shepherds Purse':11}
for i in os.listdir(path):
    label_number = label2num[i]
    new_path = path+i+'/'
    for j in fnmatch.filter(os.listdir(new_path), '*.png'):
        temp_img = image.load_img(new_path+j, target_size=(128,128))
        train_label.append(label_number)
        temp_img = image.img_to_array(temp_img)
        train_img.append(temp_img)

train_img = np.array(train_img)

train_y = pd.get_dummies(train_label)
train_y = np.array(train_y)
train_img = preprocess_input(train_img)

print('Training data shape: ', train_img.shape)
print('Training labels shape: ', train_y.shape)

/home/mzl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Training data shape:  (3550, 128, 128, 3)
Training labels shape:  (3550, 12)


In [3]:
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16

def vgg16_model(num_classes=None):

    model = VGG16(weights='imagenet', include_top=False,input_shape=(128,128,3))
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()

    model.outputs = [model.layers[-1].output]

    model.layers[-2].outbound_nodes= []
    x=Conv2D(256, kernel_size=(2,2),strides=2)(model.output)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)    
    x=Conv2D(128, kernel_size=(2,2),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x=Flatten()(x)
    x=Dense(num_classes, activation='softmax')(x)

    model=Model(model.input,x)

    for layer in model.layers[:15]:

        layer.trainable = False


    return model

In [8]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fscore(y_true, y_pred):
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    f_score = 2 * (p * r) / (p + r + K.epsilon())
    return f_score

In [9]:
from keras import backend as K
num_classes=12
model = vgg16_model(num_classes)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy',fscore])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [27]:
#Split training data into rain set and validation set
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid=train_test_split(train_img,train_y,test_size=0.1, random_state=42)

#Data augmentation
from keras.preprocessing.image import ImageDataGenerator
gen_train = ImageDataGenerator( 
    rotation_range=30,
    width_shift_range=0.2,
   height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True

)
gen_train.fit(X_train)
#Train model
from keras.callbacks import ModelCheckpoint
epochs = 20
batch_size = 64 # GPU 不好，不宜过高
model_checkpoint = ModelCheckpoint('weights8.h5', monitor='val_loss', save_best_only=True)

model.fit_generator(gen_train.flow(X_train, Y_train, batch_size=batch_size, shuffle=True), 
                    steps_per_epoch=(X_train.shape[0]//(4*batch_size)), 
                    epochs=epochs, 
                    validation_data=(X_valid,Y_valid),
                    callbacks=[model_checkpoint],verbose=1)


Epoch 1/20
12/12 [==============================] - 19s 2s/step - loss: 0.2389 - acc: 0.9167 - fscore: 0.9147 - val_loss: 0.4076 - val_acc: 0.8676 - val_fscore: 0.8710
Epoch 2/20
12/12 [==============================] - 20s 2s/step - loss: 0.1712 - acc: 0.9349 - fscore: 0.9356 - val_loss: 0.4821 - val_acc: 0.8479 - val_fscore: 0.8475
Epoch 3/20
12/12 [==============================] - 20s 2s/step - loss: 0.2085 - acc: 0.9271 - fscore: 0.9302 - val_loss: 0.4801 - val_acc: 0.8507 - val_fscore: 0.8515
Epoch 4/20
12/12 [==============================] - 19s 2s/step - loss: 0.1936 - acc: 0.9411 - fscore: 0.9433 - val_loss: 0.4016 - val_acc: 0.8732 - val_fscore: 0.8802
Epoch 5/20
12/12 [==============================] - 20s 2s/step - loss: 0.2043 - acc: 0.9271 - fscore: 0.9267 - val_loss: 0.4747 - val_acc: 0.8563 - val_fscore: 0.8579
Epoch 6/20
12/12 [==============================] - 19s 2s/step - loss: 0.2124 - acc: 0.9203 - fscore: 0.9214 - val_loss: 0.3593 - val_acc: 0.9014 - val_fscore:

In [13]:
'''
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('weight.h5', monitor='val_loss', save_best_only=True)
model.fit(X_train,Y_train,
          batch_size=32, # GPU 不好，不宜过高
          epochs=10,
          verbose=1, shuffle=True, validation_data=(X_valid,Y_valid), callbacks=[model_checkpoint])
'''

Train on 3195 samples, validate on 355 samples
Epoch 1/10
3195/3195 [==============================] - 73s 23ms/step - loss: 0.3878 - acc: 0.8707 - fscore: 0.8664 - val_loss: 3.3021 - val_acc: 0.4254 - val_fscore: 0.4040
Epoch 2/10
3195/3195 [==============================] - 66s 21ms/step - loss: 0.2485 - acc: 0.9161 - fscore: 0.9154 - val_loss: 0.9274 - val_acc: 0.7070 - val_fscore: 0.7159
Epoch 3/10
3195/3195 [==============================] - 66s 21ms/step - loss: 0.1613 - acc: 0.9512 - fscore: 0.9476 - val_loss: 0.8533 - val_acc: 0.7465 - val_fscore: 0.7404
Epoch 4/10
3195/3195 [==============================] - 66s 21ms/step - loss: 0.1234 - acc: 0.9646 - fscore: 0.9632 - val_loss: 1.6862 - val_acc: 0.6423 - val_fscore: 0.6431
Epoch 5/10
3195/3195 [==============================] - 67s 21ms/step - loss: 0.0776 - acc: 0.9772 - fscore: 0.9775 - val_loss: 0.5350 - val_acc: 0.8479 - val_fscore: 0.8491
Epoch 6/10
3195/3195 [==============================] - 65s 20ms/step - loss: 0.085

In [28]:
prob=[]
num=[]
test_img=[]
test_path = './test/'
test_all = fnmatch.filter(os.listdir(test_path), '*.png')

test_img=[]
for i in range(len(test_all)):
    path=test_path+'/'+test_all[i]
    temp_img=image.load_img(path,target_size=(128,128))
    temp_img=image.img_to_array(temp_img)
    test_img.append(temp_img) 
test_img=np.array(test_img)    
test_img=preprocess_input(test_img)


test_labels=[]
pred=model.predict(test_img)
num2label =  {0:'Loose Silky-bent', 1:'Charlock',2: 'Sugar beet',3: 'Small-flowered Cranesbill',
              4:'Common Chickweed',5: 'Common wheat',6: 'Maize', 7:'Cleavers', 8:'Scentless Mayweed',
             9: 'Fat Hen', 10:'Black-grass', 11:'Shepherds Purse'}

for i in range(len(test_all)):
    max_score =0
    lab=-1
    for j in range(12):
        if pred[i][j]>max_score:
            max_score=pred[i][j]
            lab=j
    test_labels.append(num2label[lab])


d = {'file': test_all, 'species': test_labels}
df = pd.DataFrame(data=d)
print(df.head(5))

            file           species
0  0c4199daa.png  Loose Silky-bent
1  d93c7ab6d.png       Black-grass
2  c7eb96871.png      Common wheat
3  0f6cbe5e8.png             Maize
4  eec1079a1.png          Charlock


In [29]:
#Convert dataframe to csv
df.to_csv("./submit.csv",index=False) 

#Submit the csv
print('Submitting csv')
!kg submit submit.csv -u <username> -p <password> -c plant-seedlings-classification

Submitting csv
/bin/sh: 1: cannot open username: No such file
/bin/sh: 1: kg: not found


In [31]:

model.load_weights('weight.h5')